# Demo

This notebook introduces the code's structure and functionality by demonstrating how to successfully attack various different models with appropriate (adaptive) attack strategies.

In [1]:
from collections import OrderedDict
import torch

import sys
sys.path.append("..")
import gb

In [2]:
dataset = "cora"
A, X, y = gb.data.get_dataset(dataset)
N, D = X.shape
C = y.max().item() + 1
train_nodes, val_nodes, test_nodes = gb.data.get_splits(y)[0]  # [0] = select first split

A = A.cuda()
X = X.cuda()
y = y.cuda()

## Global Attack on Vanilla GCN

This cell allows you to run evasion or poisoning attacks with different attack algorithms. At two points, you select exactly one of the provided code fragments to achieve various attacks. In any case, the found perturbation is evaluated both in the evasion and poisoning scenarios.

- "Aux-Attack" computes an evasion attack, either with the "FGA" or "PGD" optimizers.
- "Meta-Attack" computes a poisoning attack.
    - Using the "Greedy" optimizer (giving Metattack) works best with "SGD".
    - Using the "PGD" optimizer (giving Meta-PGD) works best with "Adam".

In [3]:
torch.manual_seed(42)

fit_kwargs = dict(lr=1e-2, weight_decay=5e-4)
budget = 300

def make_model():
    return gb.model.GCN(n_feat=D, n_class=C, hidden_dims=[16], dropout=0.5).cuda()

aux_model = make_model()
aux_model.fit((A, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)

def loss_fn(A_flip):
    A_pert = A + A_flip * (1 - 2 * A)

    ############### Aux-Attack ###############
    model = aux_model
    ########### Meta-Attack w/ SGD ###########
    # meta_fit_kwargs = fit_kwargs | dict(optimizer="sgd", lr=1, yield_best=False, patience=None, max_epochs=100)
    # model = make_model()
    # model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **meta_fit_kwargs, differentiable=A_pert.requires_grad)
    ########### Meta-Attack w/ Adam ##########
    # model = make_model()
    # model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs, differentiable=A_pert.requires_grad)
    ##########################################

    scores = model(A_pert, X)
    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

########### FGA for Aux-Attack ###########
# pert = gb.attack.greedy_grad_descent(A.shape, True, A.device, [budget], grad_fn, flips_per_iteration=budget, max_iterations=1)[0]
########### PGD for Aux-Attack ###########
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.1)
######### Greedy for Meta-Attack #########
# pert = gb.attack.greedy_grad_descent(A.shape, True, A.device, [budget], grad_fn, flips_per_iteration=1)[0]
########### PGD for Meta-Attack ##########
# pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.01, grad_clip=1)
##########################################

print("Clean test acc:   ", gb.metric.accuracy(aux_model(A, X)[test_nodes], y[test_nodes]).item())

A_pert = A + gb.pert.edge_diff_matrix(pert, A)
print("Adversarial edges:", pert.shape[0])
print("Evasion test acc: ", gb.metric.accuracy(aux_model(A_pert, X)[test_nodes], y[test_nodes]).item())

pois_model = make_model()
pois_model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)
print("Poisoned test acc:", gb.metric.accuracy(pois_model(A_pert, X)[test_nodes], y[test_nodes]).item())

Clean test acc:    0.8465794324874878
Adversarial edges: 299
Evasion test acc:  0.7328973412513733
Poisoned test acc: 0.7449697852134705


## Global Attack on SVD-GCN

Comment out the "w/ weights" part to observe how much our custom weighting scheme improves the attack. Like before, choose between FGA, PGD, and Meta-PGD attacks.

In [4]:
torch.manual_seed(42)

rank = 50
fit_kwargs = dict(lr=1e-2, weight_decay=5e-4)
budget = 300

def make_model():
    return gb.model.GraphSequential(OrderedDict(
        low_rank=gb.model.PreprocessA(lambda A: gb.preprocess.low_rank(A, rank)),
        gcn=gb.model.GCN(n_feat=D, n_class=C, hidden_dims=[16], dropout=0.5)
    )).cuda()

aux_model = make_model()
aux_model.fit((A, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)

A_low_rank = aux_model.low_rank(A)
A_weights = gb.metric.eigenspace_alignment(A, rank)

def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)

    ############# w/ weights #############
    A_diff = A_diff * A_weights
    ######################################

    A_pert = A_low_rank + A_diff

    ############# Aux-Attack #############
    model = aux_model.sub(exclude=["low_rank"])
    ############# Meta-Attack ############
    # model = make_model().sub(exclude=["low_rank"])
    # model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs, differentiable=A_pert.requires_grad)
    ######################################

    scores = model(A_pert, X)
    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

########### FGA for Aux-Attack ###########
# pert = gb.attack.greedy_grad_descent(A.shape, True, A.device, [budget], grad_fn, flips_per_iteration=budget, max_iterations=1)[0]
########### PGD for Aux-Attack ###########
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.1)
########### PGD for Meta-Attack ##########
# pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.1, grad_clip=0.1)
##########################################

print("Clean test acc:   ", gb.metric.accuracy(aux_model(A, X)[test_nodes], y[test_nodes]).item())

A_pert = A + gb.pert.edge_diff_matrix(pert, A)
print("Adversarial edges:", pert.shape[0])
print("Evasion test acc: ", gb.metric.accuracy(aux_model(A_pert, X)[test_nodes], y[test_nodes]).item())

pois_model = make_model()
pois_model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)
print("Poisoned test acc:", gb.metric.accuracy(pois_model(A_pert, X)[test_nodes], y[test_nodes]).item())

Clean test acc:    0.7741448283195496
Adversarial edges: 300
Evasion test acc:  0.6644868850708008
Poisoned test acc: 0.6715291738510132


## Global Attack on GNNGuard

Swap out the "w/ changed div_limit" part for "w/ real div_limit" to observe how much tuning this single hyperparameter during the attack improves its efficacy. Like before, choose between PGD and Meta-PGD attacks.

In [5]:
torch.manual_seed(42)

rank = 50
fit_kwargs = dict(lr=1e-2, weight_decay=5e-4)
budget = 300

def make_model(div_limit=1e-6):
    return gb.model.GNNGuard(n_feat=D, n_class=C, hidden_dims=[16], dropout=0.5, div_limit=div_limit).cuda()

aux_model = make_model()
aux_model.fit((A, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)

def loss_fn(A_flip):
    A_diff = A_flip * (1 - 2 * A)
    A_pert = A + A_diff

    ########## w/ real div_limit #########
    # alteration = dict()
    ######## w/ changed div_limit ########
    alteration = dict(div_limit=1e-2)
    ######################################

    ############# Aux-Attack #############
    with gb.model.changed_fields(aux_model, **alteration):
        scores = aux_model(A_pert, X)
    ############# Meta-Attack ############
    # model = make_model(**alteration)
    # model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs, max_epochs=50, differentiable=A_pert.requires_grad)
    # scores = model(A_pert, X)
    ######################################

    return gb.metric.margin(scores[test_nodes, :], y[test_nodes]).tanh().mean()

def grad_fn(A_flip):
    return torch.autograd.grad(loss_fn(A_flip), A_flip)[0]

########### PGD for Aux-Attack ###########
pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.1)
########### PGD for Meta-Attack ##########
# pert, _ = gb.attack.proj_grad_descent(A.shape, True, A.device, budget, grad_fn, loss_fn, base_lr=0.1, grad_clip=0.1)
##########################################

print("Clean test acc:   ", gb.metric.accuracy(aux_model(A, X)[test_nodes], y[test_nodes]).item())

A_pert = A + gb.pert.edge_diff_matrix(pert, A)
print("Adversarial edges:", pert.shape[0])
print("Evasion test acc: ", gb.metric.accuracy(aux_model(A_pert, X)[test_nodes], y[test_nodes]).item())

pois_model = make_model()
pois_model.fit((A_pert, X), y, train_nodes, val_nodes, progress=False, **fit_kwargs)
print("Poisoned test acc:", gb.metric.accuracy(pois_model(A_pert, X)[test_nodes], y[test_nodes]).item())

Clean test acc:    0.8390341997146606
Adversarial edges: 299
Evasion test acc:  0.7494969367980957
Poisoned test acc: 0.7525150775909424
